# 타이타닉 데이터 
  
|변수|설명|
|------|---|
|survival|Survival (0 = No; 1 = Yes)|
|pclass|Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)|
|name|Name|
|sex|Sex|
|age|Age|
|sibsp|Number of Siblings/Spouses Aboard|
|parch|Number of Parents/Children Aboard|
|ticket|Ticket Number|
|fare|Passenger Fare|
|cabin|Cabin|
|embarked|Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)|


pclass = 1,2,3등석  
sibsp = 자매 수  
parch = 부모/아이 수  
fare = 객실 요금  
cabin = 객실 번호  
embarked = 승선지  

In [1]:
# 패키지 출력
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# 타이타닉 데이터 불러오기

In [2]:
titanic = pd.read_csv('C:/Users/Edward/Desktop/PythonWorkspace/Titanic/train.csv')

# 확인
titanic.tail(2)

PassengerId  Survived  Pclass                   Name   Sex   Age  SibSp  \
889          890         1       1  Behr, Mr. Karl Howell  male  26.0      0   
890          891         0       3    Dooley, Mr. Patrick  male  32.0      0   

     Parch  Ticket   Fare Cabin Embarked  
889      0  111369  30.00  C148        C  
890      0  370376   7.75   NaN        Q

## x와 y 분리

In [3]:
target = 'Survived'
X = titanic.drop(target, axis=1)
Y = titanic[target]

## train, val, test 셋 분할

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
# test_size 속성값으로 소수점을 주면 비율로, 정수로 주면 개수로 분할해준다.
# random_state 속성값은 난수 설정값이다.
x, x_test, y, y_test = train_test_split(X, Y, test_size=.1, random_state=2022)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=80, random_state=2022)

## 결측치 확인

In [6]:
titanic.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## 결측치 채우기

### Embarked

In [14]:
from sklearn.impute import SimpleImputer
# 대상을 리스트로 선언
imputer1_list = ['Embarked']

# 선언하고 fit_transform
imputer1 = SimpleImputer(strategy = 'most_frequent')
x_train[imputer1_list] = imputer1.fit_transform(x_train[imputer1_list])
x_train.isna().sum()

c:\Users\Edward\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


PassengerId      0
Pclass           0
Name             0
Sex              0
Age            149
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          555
Embarked         0
dtype: int64

### Cabin

In [26]:
x_train['Cabin'].fillna(x_train['Cabin'].mode()[0], inplace=True)
x_train['Cabin'].isna().sum()  # 0

0

### Age

#### 가변수화

In [17]:
cat = {'Sex':['female', 'male'],
       'Embarked':['S', 'Q', 'C'],
       'Pclass':[1, 2, 3]}

for k, v in cat.items():
    x_train[k] = pd.Categorical(x_train[k], categories=v, ordered=False)
    
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 721 entries, 60 to 354
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  721 non-null    int64   
 1   Pclass       721 non-null    category
 2   Name         721 non-null    object  
 3   Sex          721 non-null    category
 4   Age          721 non-null    float64 
 5   SibSp        721 non-null    int64   
 6   Parch        721 non-null    int64   
 7   Ticket       721 non-null    object  
 8   Fare         721 non-null    float64 
 9   Cabin        166 non-null    object  
 10  Embarked     721 non-null    category
dtypes: category(3), float64(2), int64(3), object(3)
memory usage: 53.2+ KB


C:\Users\Edward\AppData\Local\Temp/ipykernel_17796/1852083886.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train[k] = pd.Categorical(x_train[k], categories=v, ordered=False)


In [19]:
cat = {'Sex':['female', 'male'],
       'Embarked':['S', 'Q', 'C'],
       'Pclass':[1, 2, 3]}
       
x_train = pd.get_dummies(x_train, columns=cat.keys(), drop_first=True)

In [20]:
x_train.tail(2)

PassengerId               Name        Age  SibSp  Parch Ticket   Fare  \
693          694   Saad, Mr. Khalil  25.000000      0      0   2672  7.225   
354          355  Yousif, Mr. Wazli  29.210962      0      0   2647  7.225   

    Cabin  Sex_male  Embarked_Q  Embarked_C  Pclass_2  Pclass_3  
693   NaN         1           0           1         0         1  
354   NaN         1           0           1         0         1

In [21]:
from sklearn.impute import KNNImputer

In [22]:
# n_neighbors=5(디폴트 값)
imputer = KNNImputer(n_neighbors=3)

In [23]:
cols = x_train.columns

In [24]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 721 entries, 60 to 354
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  721 non-null    int64  
 1   Name         721 non-null    object 
 2   Age          721 non-null    float64
 3   SibSp        721 non-null    int64  
 4   Parch        721 non-null    int64  
 5   Ticket       721 non-null    object 
 6   Fare         721 non-null    float64
 7   Cabin        166 non-null    object 
 8   Sex_male     721 non-null    uint8  
 9   Embarked_Q   721 non-null    uint8  
 10  Embarked_C   721 non-null    uint8  
 11  Pclass_2     721 non-null    uint8  
 12  Pclass_3     721 non-null    uint8  
dtypes: float64(2), int64(3), object(3), uint8(5)
memory usage: 54.2+ KB


In [27]:
x_train = imputer.fit_transform(x_train)
x_val = imputer.transform(x_val)

ValueError: could not convert string to float: 'Sirayanian, Mr. Orsen'